# LangGraph Example Overview

## What this example shows:

1. **State Management**: Uses a `TypedDict` to define the structure of data flowing through the graph
2. **Nodes**: Functions that process state (`greet_user`, `process_name`, `ask_question`)
3. **Edges**: Connections between nodes (both regular and conditional)
4. **Conditional Logic**: `should_continue` function determines the next step based on state
5. **Memory**: Uses `MemorySaver` to maintain conversation history

## Key LangGraph concepts:

- **StateGraph**: The main graph structure
- **Nodes**: Individual processing steps
- **Edges**: Flow control between nodes
- **Entry point**: Where the graph starts
- **END**: Special node to terminate the workflow

## Workflow:

This creates a simple conversational flow:
```
start → greet → process_name → ask → END
```

In [3]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Define the state structure
class State(TypedDict):
    messages: list[dict]  # List of {role: str, content: str}
    user_name: str
    step: str

# Define node functions
def greet_user(state: State) -> State:
    """First node: Greet the user"""
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": "Hello! What's your name?"}
        ],
        "step": "awaiting_name"
    }

def process_name(state: State) -> State:
    """Second node: Process user's name"""
    # Get the last user message
    user_messages = [m for m in state["messages"] if m["role"] == "user"]
    name = user_messages[-1]["content"] if user_messages else "friend"
    
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": f"Nice to meet you, {name}!"}
        ],
        "user_name": name,
        "step": "name_received"
    }

def ask_question(state: State) -> State:
    """Third node: Ask a follow-up question"""
    name = state.get("user_name", "friend")
    return {
        "messages": state["messages"] + [
            {"role": "assistant", "content": f"How can I help you today, {name}?"}
        ],
        "step": "completed"
    }

def route_next(state: State) -> str:
    """Conditional edge: Decide next step"""
    step = state.get("step", "start")
    
    if step == "start":
        return "greet"
    elif step == "awaiting_name":
        return "process_name"
    elif step == "name_received":
        return "ask"
    else:
        return "end"
    
# Build the graph
def create_chatbot():
    # Initialize the graph
    workflow = StateGraph(State)
    
    # Add nodes
    workflow.add_node("greet", greet_user)
    workflow.add_node("process_name", process_name)
    workflow.add_node("ask", ask_question)
    
    # Set entry point
    workflow.set_entry_point("greet")
    
    # Add edges
    workflow.add_edge("greet", "process_name")
    workflow.add_edge("process_name", "ask")
    workflow.add_edge("ask", END)
    
    # Compile with memory
    memory = MemorySaver()
    app = workflow.compile(checkpointer=memory)
    
    return app



In [4]:
# Create the chatbot
app = create_chatbot()

# Configuration for conversation thread
config = {"configurable": {"thread_id": "1"}}

# Initial state with user input
initial_state = {
    "messages": [
        {"role": "user", "content": "Alice"}
    ],
    "user_name": "",
    "step": "start"
}

# Run the workflow
print("Starting chatbot workflow...\n")

result = app.invoke(initial_state, config)

# Display the conversation
for msg in result['messages']:
    role = msg['role'].capitalize()
    content = msg['content']
    print(f"{role}: {content}")

print(f"\nFinal state:")
print(f"  - User name: {result['user_name']}")
print(f"  - Current step: {result['step']}")
print(f"  - Total messages: {len(result['messages'])}")


Starting chatbot workflow...

User: Alice
Assistant: Hello! What's your name?
Assistant: Nice to meet you, Alice!
Assistant: How can I help you today, Alice?

Final state:
  - User name: Alice
  - Current step: completed
  - Total messages: 4
